# Overlaying Vector Data on Raster

This notebook demonstrates how to overlay vector data (e.g., shapefiles) on raster data using `rasterio`, `geopandas`, and `matplotlib` in Python. This is useful for visualizing spatial relationships in remote sensing data, such as plotting boundaries over satellite imagery.

## Prerequisites
- Install required libraries: `rasterio`, `geopandas`, `matplotlib`, `numpy` (listed in `requirements.txt`).
- A sample GeoTIFF file (e.g., `sample.tif`) and a shapefile (e.g., `area.shp`). Replace file paths with your own data.

## Learning Objectives
- Load and visualize raster and vector data together.
- Ensure CRS alignment between raster and vector data.
- Customize the visualization for clarity.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Step 1: Load Raster and Vector Data

Load the raster file and shapefile, ensuring their coordinate reference systems (CRS) are aligned.

In [ ]:
# Define file paths
raster_path = 'sample.tif'
shapefile_path = 'area.shp'

# Load the shapefile
gdf = gpd.read_file(shapefile_path)

# Open the raster file
with rasterio.open(raster_path) as src:
    raster_data = src.read()
    raster_crs = src.crs
    raster_bounds = src.bounds

# Reproject shapefile to match raster CRS if needed
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)

# Print basic information
print(f'Raster CRS: {raster_crs}')
print(f'Shapefile CRS: {gdf.crs}')
print(f'Raster Bounds: {raster_bounds}')

## Step 2: Visualize Raster with Vector Overlay

Plot the raster (single-band or RGB) and overlay the vector geometries.

In [ ]:
# Create a plot
fig, ax = plt.subplots(figsize=(8, 8))

# Check number of bands for visualization
num_bands = raster_data.shape[0]

if num_bands == 1:
    # Single-band raster
    ax.imshow(raster_data[0], cmap='gray', extent=(raster_bounds.left, raster_bounds.right, raster_bounds.bottom, raster_bounds.top))
else:
    # Multi-band raster (RGB)
    rgb = raster_data[:3].transpose(1, 2, 0)  # Shape: (height, width, bands)
    rgb = rgb / rgb.max() if rgb.max() > 0 else rgb  # Normalize for display
    ax.imshow(rgb, extent=(raster_bounds.left, raster_bounds.right, raster_bounds.bottom, raster_bounds.top))

# Overlay vector data
gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

# Customize plot
plt.title('Raster with Vector Overlay')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

## Step 3: Visualize with Specific Vector Attributes

If the shapefile has attributes (e.g., categories), color-code the vector overlay based on an attribute.

In [ ]:
# Check if shapefile has an attribute to color by (replace 'category' with your column name)
if 'category' in gdf.columns:
    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot raster (same as above)
    if num_bands == 1:
        ax.imshow(raster_data[0], cmap='gray', extent=(raster_bounds.left, raster_bounds.right, raster_bounds.bottom, raster_bounds.top))
    else:
        rgb = raster_data[:3].transpose(1, 2, 0)
        rgb = rgb / rgb.max() if rgb.max() > 0 else rgb
        ax.imshow(rgb, extent=(raster_bounds.left, raster_bounds.right, raster_bounds.bottom, raster_bounds.top))

    # Plot vector data with color based on attribute
    gdf.plot(ax=ax, column='category', edgecolor='black', linewidth=1, legend=True)

    # Customize plot
    plt.title('Raster with Attribute-Based Vector Overlay')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()
else:
    print('No attribute named "category" found in shapefile.')

## Next Steps

- Replace `sample.tif` and `area.shp` with your own raster and shapefile.
- Adjust the attribute name (e.g., 'category') to match your shapefile’s columns.
- Experiment with different colormaps or vector styles.
- Proceed to the next notebook (`08_atmospheric_correction_snap.ipynb`) for atmospheric correction.

## Notes
- Ensure the raster and shapefile paths are correct and accessible.
- Verify CRS alignment to avoid spatial mismatches.
- Use `extent` in `imshow` to align raster and vector coordinates.
- See `docs/installation.md` for troubleshooting library installation.